In [ ]:
import numpy           as np
import pandas          as pd
from scipy.stats    import norm
from scipy.optimize import minimize, OptimizeResult

In [ ]:
it = 0
def _callback(vec):
    global it
    print(f"{it}\t {AppLoglikelihood_ALS77(np.copy(vec))}")    
    it += 1

In [ ]:
def AppEstimate_ALS77():
    
    global y, x1, x2, x3, p1, p2, p3, it
    
    alpha    = -11 
    beta1    = 0.03
    beta2    = 1.1
    beta3    = -0.01
    
    sigma2u  = 0.01
    sigma2v  = 0.0003
    lsigma2u = np.log(sigma2u)
    lsigma2v = np.log(sigma2v)
    
    theta0 = [alpha, beta1, beta2, beta3, lsigma2u, lsigma2v]
    
    print("Iteration\t f(x)")
    _callback.prev = np.copy(theta0)
    rez = minimize(AppLoglikelihood_ALS77, np.array(theta0),
                   method="Powell",
                   options={"disp": False,
                            "xtol": 1e-10,
                            "ftol": 1e-10,
                            "maxiter": 20000},
                   callback=_callback)
    it = 0
    theta  = rez.x
    logMLE = rez.fun
    
    #~~~~ standard errors ~~~~#
    theta[4:6] = np.exp(theta[4:6])
    delta = 1e-6
    grad  = pd.DataFrame( np.zeros((len(y), len(theta))) )
    
    for i in range(len(theta)):
        theta1 = np.copy(theta)
        theta1[i] += delta
        grad.iloc[:, i] = (AppLogDen_ALS77(theta1) - 
                           AppLogDen_ALS77(theta )) / delta
        
    OPG  = grad.transpose().dot(grad)
    D    = np.diag( np.concatenate(([1, 1, 1, 1], theta[4:6])) )
    ster = np.sqrt( np.diag(np.linalg.inv(OPG)) )
    
    return [theta, ster, logMLE]